### final

In [32]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select count(mgm_bldrgst_pk), sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
	(
	select distinct main_purps_cd_nm
	from m1.bld_rgst
	where main_purps_cd_nm like '%주택%'
	)
  and base_ym = '202209'
group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['bld_count','pnu_x']
df_bld_rgst['bun'] = df_bld_rgst['pnu_x'].str[:15] # 15: 본번까지 구분해서 병합
df_bld_rgst['emd_cd'] = df_bld_rgst['pnu_x'].str[:8] + '00'

# pnu별 유동인구 
sql = '''
select substring(bd_mgt_sn, 0, 20) as pnu, avg(f_tot)
  from m1.kt_bldg_xy_info
 where base_ym = '202209'
 group by substring(bd_mgt_sn, 0, 20)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['pnu','f_tot']
df_f_tot['bun'] = df_f_tot['pnu'].str[:15]


# 데이터 프레임 병합
df = pd.merge(df_bld_rgst, df_f_tot, on='bun', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['pnu'].isnull()]

# 상업용 부동산 수 / 유동인구
df['commercial_per_population'] = (df['bld_count'] / df['f_tot'])

# inf 값 제거후 컬럼 정리
df = df[~(df['commercial_per_population']==np.inf)]
df = df[['pnu','emd_cd','commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")

In [33]:
df_bld_rgst

,bld_count,pnu_x,bun,emd_cd
0,1,1111010100100030100,111101010010003,1111010100
1,1,1111010100100060003,111101010010006,1111010100
2,1,1111010100100150005,111101010010015,1111010100
3,1,1111010100100150046,111101010010015,1111010100
4,1,1111010100100300007,111101010010030,1111010100
...,...,...,...,...
2047183,1,5013032026115240006,501303202611524,5013032000
2047184,1,5013032026115240011,501303202611524,5013032000
2047185,2,5013032026115410004,501303202611541,5013032000
2047186,1,5013032026115710001,501303202611571,5013032000


In [38]:
df_f_tot

,pnu,f_tot,bun
0,1111010100100010000,2198691.000000,111101010010001
1,1111010100100030065,3988060.000000,111101010010003
2,1111010100100060003,1444137.000000,111101010010006
3,1111010100100150005,2198691.000000,111101010010015
4,1111010100100150009,2198691.000000,111101010010015
...,...,...,...
1339480,5013032026108430003,97462.000000,501303202610843
1339481,5013032026113380001,37262.000000,501303202611338
1339482,5013032026114350009,28705.000000,501303202611435
1339483,5013032026114810001,37262.000000,501303202611481


In [37]:
df

,pnu,emd_cd,commercial_per_population,base_ym
0,1111010100100030065,1111010100,0.000000,202211
1,1111010100100060003,1111010100,0.000001,202211
2,1111010100100150005,1111010100,0.000000,202211
3,1111010100100150009,1111010100,0.000000,202211
4,1111010100100150056,1111010100,0.000000,202211
...,...,...,...,...
9734808,5013032026115240008,5013032000,0.000010,202211
9734809,5013032026115240006,5013032000,0.000010,202211
9734810,5013032026115240001,5013032000,0.000010,202211
9734811,5013032026115240008,5013032000,0.000010,202211


In [41]:
df['commercial_per_population'].median()

2.8820295944090933e-07

In [39]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-03 17:58:27.596933  :  0
2022-11-03 17:58:30.516045  :  10000
2022-11-03 17:58:32.801245  :  20000
2022-11-03 17:58:34.745037  :  30000
2022-11-03 17:58:36.975171  :  40000
2022-11-03 17:58:39.353799  :  50000
2022-11-03 17:58:41.864627  :  60000
2022-11-03 17:58:44.082782  :  70000
2022-11-03 17:58:49.196409  :  80000
2022-11-03 17:58:53.269273  :  90000
2022-11-03 17:59:01.542814  :  100000
2022-11-03 17:59:10.506096  :  110000
2022-11-03 17:59:16.758632  :  120000
2022-11-03 17:59:33.648532  :  130000
2022-11-03 17:59:36.691538  :  140000
2022-11-03 17:59:39.126525  :  150000
2022-11-03 17:59:41.640692  :  160000
2022-11-03 17:59:43.949140  :  170000
2022-11-03 17:59:46.018814  :  180000
2022-11-03 17:59:50.053839  :  190000
2022-11-03 17:59:55.956637  :  200000
2022-11-03 17:59:58.991098  :  210000
2022-11-03 18:00:01.018952  :  220000
2022-11-03 18:00:06.551956  :  230000
2022-11-03 18:00:11.884712  :  240000
2022-11-03 18:00:14.051411  :  250000
2022-11-03 18:00:18.980303

In [83]:
df.describe()

,commercial_per_population
count,462282.000000
mean,0.000857
std,0.032387
min,0.000000
25%,0.000007
50%,0.000015
75%,0.000057
max,18.000000


###### ver1 (법정동 단위)

In [38]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 법정동별 상가 면적
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']
df_f_tot = df_f_tot[~df_f_tot['f_tot'].isnull()].reset_index(drop=True)

# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 상가면적 / 거주인구
df['commercial_per_population'] = (df['total_count'] / df['f_tot'])
df = df[['emd_cd','commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_commercial_per_population;
    create table m2.datadam_commercial_per_population (    
    emd_cd varchar(10)
    , commercial_per_population double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 18:08:47.688140  :  0
2022-10-30 18:08:50.007249  :  10000
17296


In [37]:
df.describe()

,commercial_per_population
count,17296.000000
mean,0.051612
std,0.292162
min,0.000000
25%,0.000259
50%,0.003113
75%,0.020263
max,17.100000
